In [148]:
from deep_translator.exceptions import RequestError
from deep_translator import GoogleTranslator

import pandas as pd
import unicodedata
import dateparser
import numpy as np
import ast
import re
import nltk
from tqdm import tqdm
import pickle
import re

In [123]:
def en_ru(text):
    try:
        return GoogleTranslator(source='en', target='ru').translate(text)
    except Exception as e:
        return ''

def ru_en(text):
    try:
        return GoogleTranslator(source='ru', target='en').translate(text)
    except Exception as e:
        return ''

def en_ru_ext(text):
    try:
        sentences = nltk.tokenize.sent_tokenize(text)
        trans_sentences = ' '.join([str(en_ru(i)) for i in sentences])
        return trans_sentences
    except Exception as e:
        return ''

In [81]:
names = [
    'silkwaychina', #+
    'prc_roday', #+
    'russian.news.cn', #+
    'businessemirates', #+
    'india-briefing', #+
    'vietnam-briefing', #+
    'middleeastbriefing', #+
    'indianstartupnews', #+
    'mea-markets', #+
    'agbi', 
    'chinadaily' #+
]

### Обработка данных

In [82]:
data_all = pd.DataFrame()

In [83]:
postfix = "with_tags"

#### По источникам

In [84]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'silkwaychina'
country = 'China'
language = 'ru'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language
data.text = data.text.apply(lambda value: value.replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data['date'] = data['date'].apply(lambda value: dateparser.parse(value, languages=['ru']).date())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,silkwaychina,China,ru,Торговля на OZON и Wildberries: Как начать про...,2023-12-27,https://silkwaychina.ru/news/news/statya/torgo...,В данной статье мы рассмотрим ключевые аспекты...,<p> В данной статье мы рассмотрим ключевые асп...
0,silkwaychina,China,ru,Оптимизация процесса доставки товаров с 1688 д...,2023-12-20,https://silkwaychina.ru/news/news/statya/optim...,В данной статье рассмотрим практические советы...,<p>В данной статье рассмотрим практические сов...
0,silkwaychina,China,ru,Китайский Новый год: подготовьтесь к праздничн...,2023-12-19,https://silkwaychina.ru/news/news/aktualnoe/ki...,"Скоро начинается Китайский Новый год, а это зн...","<p>Скоро начинается Китайский Новый год, а это..."
0,silkwaychina,China,ru,Сравнительный анализ: преимущества и недостатк...,2023-12-15,https://silkwaychina.ru/news/news/statya/sravn...,В современном мире онлайн-торговли одним из кл...,<p>В современном мире онлайн-торговли одним из...
0,silkwaychina,China,ru,Секреты работы с 1688: Как найти качественные ...,2023-12-11,https://silkwaychina.ru/news/news/statya/sekre...,В этой статье мы раскроем секреты работы с пла...,<p>В этой статье мы раскроем секреты работы с ...


(79, 8)


In [85]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'prc_roday'
country = 'China'
language = 'ru'

# Новости из тг канала, поэтому нет версии с тэгами

data = pd.read_excel(f'{name}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['date'] = pd.to_datetime(data['date'])
data['country'] = country
data['language'] = language
data['text_with_tags'] = np.nan
data.text = data.text.apply(lambda value: value.replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,prc_roday,China,ru,Экономика Китая устойчива в первом квартале,2024-04-03 16:41:43,https://www.india-briefing.com/news/top-sector...,Экономика Китая приветствовала уверенный старт...,NaN
0,prc_roday,China,ru,Добыча угля в Китае достигла исторического мак...,2024-04-03 14:40:27,https://www.india-briefing.com/news/top-sector...,Добыча угля в Китае: объемы производства и имп...,NaN
0,prc_roday,China,ru,"Компания Huawei заявляет о росте выручки на 9,...",2024-04-03 12:39:37,https://www.india-briefing.com/news/top-sector...,"Китайская компания Huawei сообщила 29 марта, ч...",NaN
0,prc_roday,China,ru,Немецкая оптическая компания ZEISS открывает ц...,2024-04-03 10:39:21,https://www.india-briefing.com/news/top-sector...,"ZEISS, немецкая транснациональная компания, ра...",NaN
0,prc_roday,China,ru,Качественное развитие китайской экономики – бл...,2024-04-03 08:43:59,https://www.india-briefing.com/news/top-sector...,Развитие китайской экономики: Высококачественн...,NaN


(1529, 8)


In [86]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'russian.news.cn'
country = 'China'
language = 'ru'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['date'] = pd.to_datetime(data['date'])
data['country'] = country
data['language'] = language

result_list = []

for value in data.text:
    if len(value.split('--')) > 1:
        result = value.split('--')[1].replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip()
    else:
        result = value.replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip()
        
    result_list.append(result)

data.text = result_list

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,russian.news.cn,China,ru,Китайская робототехническая компания и FAW-Vol...,2024-07-02 22:21:31,https://russian.news.cn/20240702/226c90aab40e4...,"Компания UBTECH, занимающаяся исследованиями и...",<h1>\r\n Ви...
0,russian.news.cn,China,ru,Китай и Швейцария начнут переговоры по обновле...,2024-07-02 19:23:31,https://russian.news.cn/20240702/68906c8c41c84...,Китай и Швейцария в понедельник подписали мемо...,<h1>\r\n Ви...
0,russian.news.cn,China,ru,В июне 2024 года китайский логистический секто...,2024-07-02 19:23:30,https://russian.news.cn/20240702/671257fe766d4...,В июне этого года в логистическом секторе Кита...,<h1>\r\n Ви...
0,russian.news.cn,China,ru,"В Китае начинает работу первый в стране ""зелен...",2024-07-02 19:08:30,https://russian.news.cn/20240702/a78a1c50a8204...,Китайская национальная офшорная нефтяная корпо...,<h1>\r\n Ви...
0,russian.news.cn,China,ru,Новый авиамаршрут связал Шанхай с французским ...,2024-07-02 17:06:30,https://russian.news.cn/20240702/72c0bdcb12b84...,Самолет с более чем 290 пассажирами на борту в...,<h1>\r\n Ви...


(164, 8)


In [87]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'businessemirates'
country = 'UAE'
language = 'ru'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language
data.text = data.text.apply(lambda value: value.replace('\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n\n\n', '').replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data['date'] = data['date'].apply(lambda value: dateparser.parse(value))

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,businessemirates,UAE,ru,Власти Шарджи объявили о создании новой свобод...,2024-02-07 16:40:00,https://businessemirates.ae/news/uae-property-...,Власти Шарджи объявили о создании новой свобод...,<p></p><p>Власти Шарджи объявили о создании но...
0,businessemirates,UAE,ru,Власти Дубая утвердили Программу увеличения ПИИ,2024-02-07 16:00:00,https://businessemirates.ae/news/uae-property-...,Исполнительный совет Дубая утвердил Программу ...,<p></p><p>Исполнительный совет Дубая утвердил ...
0,businessemirates,UAE,ru,​Страны ОПЕК+ продлили добровольные сокращения...,2024-02-07 06:30:00,https://businessemirates.ae/news/uae-property-...,Ряд стран – участниц соглашения ОПЕК+ продолжа...,"<p></p><p><span style=""background-color: initi..."
0,businessemirates,UAE,ru,Экономику ОАЭ ожидает дальнейший рост во второ...,2024-01-07 07:40:00,https://businessemirates.ae/news/uae-property-...,Во второй половине текущего года экономику ОАЭ...,<p></p><p>Во второй половине текущего года эко...
0,businessemirates,UAE,ru,"Дом в Дубае продан за 240,5 млн дирхамов",2024-01-07 07:10:00,https://businessemirates.ae/news/uae-property-...,"Вилла с шестью спальнями, находящаяся на остро...","<p></p><p>Вилла с шестью спальнями, находящаяс..."


(399, 8)


In [88]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'india-briefing'
country = 'India'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].dt.tz_localize(None)
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,india-briefing,India,eng,India’s Standing in the Emerging Asia Manufact...,2024-06-27 14:43:05,https://www.india-briefing.com/news/india-stan...,According to the Emerging Asia Manufacturing I...,<p><strong><em>According to the Emerging Asia ...
0,india-briefing,India,eng,FDI in India FY 2023-24: Top State and Sector ...,2024-06-26 15:37:11,https://www.india-briefing.com/news/fdi-landsc...,We discuss which Indian states and sectors rec...,<p><strong><em>We discuss which Indian states ...
0,india-briefing,India,eng,India to Propel South Asia as Fastest Growing ...,2024-06-13 20:17:36,https://www.india-briefing.com/news/india-to-p...,The World Bank projects that the Indian econom...,<p><em><strong>The World Bank projects that th...
0,india-briefing,India,eng,India-Qatar Investment and Trade Relations,2024-06-13 19:30:33,https://www.india-briefing.com/news/india-qata...,India and Qatar have built a robust investment...,<p>India and Qatar have built a robust investm...
0,india-briefing,India,eng,Ministerial Roles in Modi’s New Cabinet: Full ...,2024-06-11 10:30:14,https://www.india-briefing.com/news/modi-2024-...,Bharatiya Janata Party (BJP) leader Narendra M...,<p><em><strong>Bharatiya Janata Party (BJP) le...


(261, 8)


In [89]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'vietnam-briefing'
country = 'Vietnam'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].dt.tz_localize(None)
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,vietnam-briefing,Vietnam,eng,Vietnam Calls for Chinese Support in Advancing...,2024-07-02 15:36:52,https://www.vietnam-briefing.com/news/vietnam-...,The Vietnam government is seeking more Chinese...,<p><em><strong>The Vietnam government is seeki...
0,vietnam-briefing,Vietnam,eng,Trump v Biden: How Will the 2024 US Presidenti...,2024-06-27 20:12:24,https://www.vietnam-briefing.com/news/trump-v-...,With both Donald Trump and sitting president J...,<p><strong><em>With both Donald Trump and sitt...
0,vietnam-briefing,Vietnam,eng,Vietnam-Mexico Trade and Investment Relations,2024-06-21 19:46:21,https://www.vietnam-briefing.com/news/vietnam-...,Vietnam is Mexico’s second-largest trading par...,<p>Vietnam is Mexico’s second-largest trading ...
0,vietnam-briefing,Vietnam,eng,EU Updates Food Safety Measures for Vietnam Ag...,2024-06-19 18:57:45,https://www.vietnam-briefing.com/news/eu-updat...,We note the updated EU food safety regulations...,<p><strong><em>We note the updated EU food saf...
0,vietnam-briefing,Vietnam,eng,Vietnam-U.S. Trade and Investment Relations,2024-06-17 18:09:24,https://www.vietnam-briefing.com/news/vietnam-...,Since Vietnam and the U.S. established diploma...,<p><strong><em>Since Vietnam and the U.S. esta...


(138, 8)


In [90]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'middleeastbriefing'
country = 'Middle East'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].dt.tz_localize(None)
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,middleeastbriefing,Middle East,eng,Facilitating Trade and Connectivity between UA...,2024-07-01 10:25:09,https://www.middleeastbriefing.com/news/facili...,In a significant move aimed at enhancing trade...,<p>In a significant move aimed at enhancing tr...
0,middleeastbriefing,Middle East,eng,Abu Dhabi Courting Major UK Hedge Funds,2024-06-19 12:14:48,https://www.middleeastbriefing.com/news/abu-dh...,Abu Dhabi is actively courting two major UK he...,<p>Abu Dhabi is actively courting two major UK...
0,middleeastbriefing,Middle East,eng,Dubai-Based Ports and Logistics Giant DP World...,2024-06-19 06:20:06,https://www.middleeastbriefing.com/news/dp-wor...,DP World is set to invest $3 billion in new po...,<p>DP World is set to invest $3 billion in new...
0,middleeastbriefing,Middle East,eng,QIA Reported to Acquire 10 percent Stake in Ch...,2024-06-12 11:11:54,https://www.middleeastbriefing.com/news/qia-re...,"Qatar’s sovereign wealth fund, QIA, is acquiri...","<p><em><strong>Qatar’s sovereign wealth fund, ..."
0,middleeastbriefing,Middle East,eng,Choosing Dubai as Your Investment Base and Set...,2024-06-11 12:56:25,https://www.middleeastbriefing.com/news/choosi...,"Based in Dubai, IFZA is a dynamic internationa...","<p><em><strong>Based in Dubai, IFZA is a dynam..."


(402, 8)


In [91]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'indianstartupnews'
country = 'India'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = pd.to_datetime(data['date'])
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

C:\Users\User\AppData\Local\Temp\ipykernel_15224\887056562.py:18: FutureWarning: Parsed string "02 Jul 2024 18:28 IST" included an un-recognized timezone "IST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  data['date'] = pd.to_datetime(data['date'])


,source,country,language,title,date,href,text,text_with_tags
0,indianstartupnews,India,eng,Peak XV-backed AI-health startup Cloudphysicia...,NaT,/news/peak-xv-backed-ai-health-startup-cloudph...,Bengaluru-based AI healthcare startup Cloudphy...,<p><span>Bengaluru-based AI healthcare startup...
0,indianstartupnews,India,eng,Air India to set up South Asia's largest flyin...,2024-07-02 18:28:00,/news/air-india-to-set-up-south-asias-largest-...,Tata Group's airline unit Air India is set to ...,<p>Tata Group's airline unit Air India is set ...
0,indianstartupnews,India,eng,Customised ayurveda beauty brand Vedix partner...,2024-07-02 13:48:00,/news/customised-ayurveda-beauty-brand-vedix-p...,"Vedix, a customised Ayurvedic personal care br...","<p>Vedix, a customised Ayurvedic personal care..."
0,indianstartupnews,India,eng,"RBI, ASEAN countries to create platform to fac...",2024-07-02 12:38:00,/news/rbi-asean-countries-to-create-platform-t...,The Reserve Bank of India (RBI) has announced ...,<p>The Reserve Bank of India (RBI) has announc...
0,indianstartupnews,India,eng,IBM signs MoU with Gujarat govt to establish A...,2024-07-01 21:40:00,/news/ibm-signs-mou-with-gujarat-govt-to-estab...,US-based technology company IBM and the Depart...,<p>US-based technology company IBM and the Dep...


(441, 8)


In [92]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'mea-markets'
country = 'UAE'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = data.date.apply(lambda value: pd.to_datetime(' '.join(value.replace(',', '').split(' ')[2:])))
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,mea-markets,UAE,eng,How South African Retailers Can Capitalise on ...,2024-07-02,https://www.mea-markets.com/how-south-african-...,"For many retailers, winter is one of the weake...","<p>For many retailers, winter is one of the we..."
0,mea-markets,UAE,eng,Why the Promise of a Cashless Society is Key t...,2024-06-28,https://www.mea-markets.com/why-the-promise-of...,"By Justin Floyd, CEO of RedcloudUnder the curr...","<p class=""has-text-align-center""><strong><em>B..."
0,mea-markets,UAE,eng,Hidden GEMs: Africa’s Frontier Markets Have Fu...,2024-06-27,https://www.mea-markets.com/hidden-gems-africa...,African economies have rebounded in recent mon...,<p>African economies have rebounded in recent ...
0,mea-markets,UAE,eng,Why Nigeria’s Maritime Surveillance Systems Ne...,2024-06-13,https://www.mea-markets.com/why-nigerias-marit...,Nigeria will benefit much more if all agencies...,"<p class=""has-text-align-center""><strong><em>N..."
0,mea-markets,UAE,eng,UAE Committed to Reducing Carbon Emissions in ...,2024-06-11,https://www.mea-markets.com/uae-committed-to-r...,In the path of achieving net-zero carbon emiss...,<p><strong>In the path of achieving net-zero c...


(246, 8)


In [93]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'agbi'
country = 'Arabian'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = data.date.apply(lambda value: pd.to_datetime(value.replace('\t', '').replace('\n', '')))
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,agbi,Arabian,eng,My election forecast: UK nationals in Gulf sho...,2024-07-02,https://www.agbi.com/opinion/finance/2024/07/b...,“At last we can return to sanity ... the mad h...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,Arabian,eng,Saudi forex reserves at 18-month high in boost...,2024-07-02,https://www.agbi.com/banking-finance/2024/07/s...,Saudi Arabia’s foreign exchange reserves have ...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,Arabian,eng,Abu Dhabi court rulings strengthen UAE arbitra...,2024-07-02,https://www.agbi.com/analysis/employment/2024/...,Recent court rulings in Abu Dhabi further tigh...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,Arabian,eng,"UAE to axe 2,000 procedures in ‘Zero Bureaucra...",2024-07-02,https://www.agbi.com/banking-finance/2024/07/u...,The Central Bank of the UAE has announced plan...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,Arabian,eng,GCC economic data,2024-07-02,https://www.agbi.com/economy/2024/07/gcc-econo...,The charts and tables on this page are for gen...,"<li class=""social"">\n<a class=""social__link so..."


(1067, 8)


In [94]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'chinadaily'
country = 'China'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date']  = data['date'].apply(lambda value: value.split('Updated: ')[1].replace('\n', '').lstrip().rstrip())
data['date'] = pd.to_datetime(data['date'])
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,chinadaily,China,eng,State Grid's efforts power up Suzhou's photovo...,2024-07-02 16:30:00,https://www.chinadaily.com.cn/a/202407/02/WS66...,The State Grid Suzhou City Suburban Power Supp...,<p>The State Grid Suzhou City Suburban Power S...
0,chinadaily,China,eng,Summer peak load alleviated by State Grid proj...,2024-07-02 16:28:00,https://www.chinadaily.com.cn/a/202407/02/WS66...,"On June 26, the Bengbu Toupu 220 kilovolt tran...","<p>On June 26, the Bengbu Toupu 220 kilovolt t..."
0,chinadaily,China,eng,PayPal bullish on nation's opening-up,2024-07-02 09:21:00,https://www.chinadaily.com.cn/a/202407/02/WS66...,China's commitment to further open up its fina...,<p>China's commitment to further open up its f...
0,chinadaily,China,eng,Green compliant oilfield put into operation,2024-07-02 09:17:00,https://www.chinadaily.com.cn/a/202407/02/WS66...,China's first comprehensive environmentally fr...,<p>China's first comprehensive environmentally...
0,chinadaily,China,eng,Electricity supplies put to the test ahead of ...,2024-07-01 23:41:00,https://www.chinadaily.com.cn/a/202407/01/WS66...,"At a senior high school in Xiangcheng, a count...","<p>At a senior high school in Xiangcheng, a co..."


(753, 8)


In [95]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'gbm.hsbc'
country = 'China'
language = 'eng'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = pd.to_datetime(data['date'])
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

data.head()

,source,country,language,title,date,href,text,text_with_tags
0,gbm.hsbc,China,eng,HSBC's Business Notice Account solution optimi...,2024-06-18,https://www.gbm.hsbc.com/en-gb/insights/market...,AJ Bell is one of the largest investment platf...,<p>AJ Bell is one of the largest investment pl...
0,gbm.hsbc,China,eng,Reconfiguring globalisation in a new world order,2024-06-17,https://www.gbm.hsbc.com/en-gb/insights/intern...,Globalisation is under threat. In an increasin...,<p>Globalisation is under threat. In an increa...
0,gbm.hsbc,China,eng,Spotlight on India: Factors driving the markets,2024-06-17,https://www.gbm.hsbc.com/en-gb/insights/market...,India’s recognition as the world’s fastest-gro...,<p>India’s recognition as the world’s fastest-...
0,gbm.hsbc,China,eng,Thriving in the never normal,2024-06-17,https://www.gbm.hsbc.com/en-gb/insights/market...,The post-pandemic world did not return to the ...,<p>The post-pandemic world did not return to t...
0,gbm.hsbc,China,eng,ASEAN – a region on the rise,2024-06-17,https://www.gbm.hsbc.com/en-gb/insights/market...,The Association of Southeast Asian Nations (AS...,<p>The Association of Southeast Asian Nations ...


In [96]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'inc42'
country = 'China'
language = 'eng'

#Нет тэгов, так как источник заново не парсится

data = pd.read_excel(f'{name}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language
data['text_with_tags'] = np.nan
data['date'] = pd.to_datetime(data['date'], format="%dth %B, %Y")
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,inc42,China,eng,Need To Harness AI For Indian Youth Despite Ch...,2024-05-18,https://inc42.com/buzz/need-to-harness-ai-for-...,Chief economic advisor (CEA) V Anantha Nageswa...,NaN
0,inc42,China,eng,Urban Company’s CEO Expects UAE Operations To ...,2024-05-18,https://inc42.com/buzz/urban-companys-ceo-expe...,Just days after Urban Company’s cofounder and ...,NaN
0,inc42,China,eng,IPO-Bound Ola Electric In Discussions With Kay...,2024-05-18,https://inc42.com/buzz/ipo-bound-ola-electric-...,Electric vehicle maker Ola Electric is reporte...,NaN
0,inc42,China,eng,From Propelld To RED.Health — Indian Startups ...,2024-05-18,https://inc42.com/buzz/from-propelld-to-red-he...,Funding activity in the world’s third largest ...,NaN
0,inc42,China,eng,India’s AI Regulations Won’t Deter Innovation:...,2024-05-18,https://inc42.com/buzz/indias-ai-regulations-w...,The Centre plans to regulate artificial intell...,NaN


(17, 8)


In [97]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'kitau'
country = 'China'
language = 'ru'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data['date'] = data['date'].fillna('1970-01-01')

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,kitau,China,ru,Контейнерный кризис 2.0: почему растут ставк...,1970-01-01,https://kitau.ru/stati/proizvodstvo/konteynern...,Мировая торговля более чем на 90% зависит от м...,<p>\r\n\t Мировая торговля более чем на 90% за...
0,kitau,China,ru,Валютный контроль по внешнеторговым контракта...,1970-01-01,https://kitau.ru/stati/proizvodstvo/valyutnyy-...,С апреля 2024 года в России вступили в силу но...,<p>\r\n\t С апреля 2024 года в России вступили...
0,kitau,China,ru,Что значит «потерять лицо» в Китае и почему эт...,1970-01-01,https://kitau.ru/stati/proizvodstvo/chto-znach...,Бизнес с Китаем это не только про деньги. Успе...,<p>\r\n\t Бизнес с Китаем это не только про де...
0,kitau,China,ru,Какие банки Китая принимают платежи из России,1970-01-01,https://kitau.ru/stati/proizvodstvo/kakie-bank...,С февраля перечислять деньги в Китай стало го...,<p>\n<br/>\n</p><p>\r\n\t С февраля перечислят...
0,kitau,China,ru,Как зарегистрировать компанию в Пекине через W...,1970-01-01,https://kitau.ru/stati/proizvodstvo/kak-zaregi...,В Пекине насчитывается около 35 тысяч иностран...,<p>\r\n\t В Пекине насчитывается около 35 тыся...


(44, 8)


In [98]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'as-pacific'
country = 'China'
language = 'ru'

data = pd.read_excel(f'{name}{postfix}.xlsx', index_col=0)
data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language

data['date'] = data['date'].apply(lambda value: dateparser.parse(value, languages=['ru']).date())
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,as-pacific,China,ru,Китайские шопинг-фестивали 2022-2023,2023-08-03,https://as-pacific.com/blog/category-vystavki-...,Китайский Новый год или Праздник весны (春节)...,"['<li class=""breadcrumb-item"" itemprop=""itemLi..."
0,as-pacific,China,ru,Как провести лайвстрим-шопинг в Китае,2023-08-01,https://as-pacific.com/blog/kak-provesti-shopi...,"В этой статье мы расскажем вам о том, как про...","['<li class=""breadcrumb-item"" itemprop=""itemLi..."
0,as-pacific,China,ru,Zhihu или китайская Quora - как использовать ...,2023-04-03,https://as-pacific.com/blog/category-kitayskiy...,На сегодняшний день площадка для вопросов и о...,"['<li class=""breadcrumb-item"" itemprop=""itemLi..."
0,as-pacific,China,ru,Экосистема Alibaba - набор сервисов для повыше...,2023-03-21,https://as-pacific.com/blog/category-vse-o-biz...,Сегодня экосистема Алибаба – одна из крупнейш...,"['<li class=""breadcrumb-item"" itemprop=""itemLi..."
0,as-pacific,China,ru,Экономические тренды Китая,2023-03-16,https://as-pacific.com/blog/category-vse-o-biz...,Несмотря на стремительный экономический рост...,"['<li class=""breadcrumb-item"" itemprop=""itemLi..."


(40, 8)


In [99]:
cols = ['source', 'country', 'language', 'title', 'date', 'href', 'text', 'text_with_tags']

name = 'skolkovo'
country = 'China'
language = 'ru'

#Нет тэгов, так как статей мало, копировались в ручную

data = pd.read_excel(f'{name}.xlsx')

data = data[data['title']!='']
data = data[data['href']!='']
data = data[data['text']!='']

data = data[~data.title.isna()]
data = data[~data.href.isna()]
data = data[~data.text.isna()]
data['country'] = country
data['language'] = language
data['text_with_tags'] = np.nan
data['date'] = data['date'].apply(lambda value: dateparser.parse(value, languages=['ru']).date())
data.text = data.text.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())
data.title = data.title.apply(lambda value: unicodedata.normalize("NFKD", value).replace('\r', '').replace('\n', '').replace('\t', '').rstrip().lstrip())

data = data[cols]

data_all = pd.concat([data_all, data], axis=0)

display(data.head())

print(data.shape)

,source,country,language,title,date,href,text,text_with_tags
0,skolkovo,China,ru,Законодательство Китая: юридические «подводные...,2023-08-10,https://www.skolkovo.ru/expert-opinions/zakono...,Законодательство Китая: юридические «подводные...,NaN
1,skolkovo,China,ru,Что нужно знать о создании своего бизнеса в Китае,2023-07-31,https://www.skolkovo.ru/expert-opinions/chto-n...,Особенности взаимодействия бизнес-культурКак ...,NaN
2,skolkovo,China,ru,Особенности продвижения продукта в Китае,2023-07-14,https://www.skolkovo.ru/expert-opinions/osoben...,О нюансах маркетинга рассказали Валерий Кайг...,NaN
3,skolkovo,China,ru,Китай как глобальный партнер: движущие силы ...,2023-07-13,https://www.skolkovo.ru/expert-opinions/kitaj-...,Первая серия эфиров посвящена Китаю. Проректор...,NaN
4,skolkovo,China,ru,Бизнес в Индии: юридические сложности и как их...,2023-10-24,https://www.skolkovo.ru/expert-opinions/biznes...,На чем основана правовая система ИндииВо-первы...,NaN


(12, 8)


### Постобработка

In [100]:
data_all

,source,country,language,title,date,href,text,text_with_tags
0,silkwaychina,China,ru,Торговля на OZON и Wildberries: Как начать про...,2023-12-27,https://silkwaychina.ru/news/news/statya/torgo...,В данной статье мы рассмотрим ключевые аспекты...,<p> В данной статье мы рассмотрим ключевые асп...
0,silkwaychina,China,ru,Оптимизация процесса доставки товаров с 1688 д...,2023-12-20,https://silkwaychina.ru/news/news/statya/optim...,В данной статье рассмотрим практические советы...,<p>В данной статье рассмотрим практические сов...
0,silkwaychina,China,ru,Китайский Новый год: подготовьтесь к праздничн...,2023-12-19,https://silkwaychina.ru/news/news/aktualnoe/ki...,"Скоро начинается Китайский Новый год, а это зн...","<p>Скоро начинается Китайский Новый год, а это..."
0,silkwaychina,China,ru,Сравнительный анализ: преимущества и недостатк...,2023-12-15,https://silkwaychina.ru/news/news/statya/sravn...,В современном мире онлайн-торговли одним из кл...,<p>В современном мире онлайн-торговли одним из...
0,silkwaychina,China,ru,Секреты работы с 1688: Как найти качественные ...,2023-12-11,https://silkwaychina.ru/news/news/statya/sekre...,В этой статье мы раскроем секреты работы с пла...,<p>В этой статье мы раскроем секреты работы с ...
...,...,...,...,...,...,...,...,...
7,skolkovo,China,ru,Открывая Индию: феномен развития и перспективы...,2023-10-02,https://www.skolkovo.ru/expert-opinions/otkryv...,Особенности культурно-исторического развитияГл...,NaN
8,skolkovo,China,ru,Африка с точки зрения бизнеса: фабрика человеч...,2024-02-19,https://www.skolkovo.ru/expert-opinions/afrika...,Многообразие – ключевая загадка АфрикиЕсли мы ...,NaN
9,skolkovo,China,ru,Почему российскому бизнесу стоит идти в Африк...,2024-01-26,https://www.skolkovo.ru/expert-opinions/pochem...,Многие привыкли рассуждать об Африке как о чем...,NaN
10,skolkovo,China,ru,Неочевидные регионы: где сегодня работать комп...,2022-10-31,https://www.skolkovo.ru/expert-opinions/neoche...,Ситуация в мировой экономике привела к ограни...,NaN


In [101]:
data_all['date'] = pd.to_datetime(data_all['date'])
data_all['year'] = data_all['date'].dt.year

In [102]:
data_all.source.unique()

array(['silkwaychina', 'prc_roday', 'russian.news.cn', 'businessemirates',
       'india-briefing', 'vietnam-briefing', 'middleeastbriefing',
       'indianstartupnews', 'mea-markets', 'agbi', 'chinadaily',
       'gbm.hsbc', 'inc42', 'kitau', 'as-pacific', 'skolkovo'],
      dtype=object)

In [103]:
data_all.source.nunique()

16

In [104]:
data_all.groupby(['source'], as_index=False).text.count()

,source,text
0,agbi,1067
1,as-pacific,40
2,businessemirates,399
3,chinadaily,753
4,gbm.hsbc,193
5,inc42,17
6,india-briefing,261
7,indianstartupnews,441
8,kitau,44
9,mea-markets,246


In [105]:
data_all.groupby(['year'], as_index=False).text.count()

,year,text
0,1970.0,44
1,2015.0,19
2,2016.0,47
3,2017.0,6
4,2018.0,13
5,2019.0,38
6,2020.0,61
7,2021.0,97
8,2022.0,221
9,2023.0,2379


In [106]:
data_all.groupby(['source', 'year'], as_index=False).text.count()

,source,year,text
0,agbi,2023.0,329
1,agbi,2024.0,738
2,as-pacific,2021.0,7
3,as-pacific,2022.0,22
4,as-pacific,2023.0,11
5,businessemirates,2023.0,168
6,businessemirates,2024.0,231
7,chinadaily,2023.0,121
8,chinadaily,2024.0,632
9,gbm.hsbc,2022.0,53


In [107]:
print(data_all.shape)
data_all = data_all[data_all['year'].isin([1970, 2023, 2024])]
print(data_all.shape)

(5785, 9)
(5282, 9)


In [108]:
print(data_all.shape)
data_all = data_all.drop_duplicates()
print(data_all.shape)

(5282, 9)
(5242, 9)


In [111]:
info_dict = {
    'silkwaychina': ['insights', 'business'],
    'prc_roday': ['news', 'business'],
    'russian.news.cn': ['news', 'economic'],
    'businessemirates': ['news', 'business'],
    'india-briefing': ['news', 'economic'],
    'vietnam-briefing': ['news', 'economic'],
    'middleeastbriefing': ['news', 'economic'],
    'indianstartupnews': ['news', 'business'],
    'mea-markets': ['news', 'business'],
    'agbi': ['news', 'finance'],
    'chinadaily': ['news', 'business'],
    'gbm.hsbc': ['insights', 'business'],
    'inc42': ['news', 'all'],
    'as-pacific': ['insights', 'business'],
    'skolkovo': ['insights', 'business'],
    'kitau': ['insights', 'business']
}


data_all['type'] = data_all.source.apply(lambda value: info_dict[value][0])
data_all['category'] = data_all.source.apply(lambda value: info_dict[value][1])

In [ ]:
#Переводим по предложениям, чтобы избежать вылета на длинных текстах. Но это занимает больше времени.
#Можно скоратить время значительно переводя сначала без деления на предложения. А потом остатки допереводить по предложениям.

In [116]:
%%time

data_all['title_ru'] = data_all.apply(lambda row: en_ru_ext(row['title']) if row['language']=='eng' else row['title'], axis=1)

CPU times: total: 7min 59s
Wall time: 28min 36s


In [117]:
%%time

data_all['text_ru'] = data_all.apply(lambda row: en_ru_ext(row['text']) if row['language']=='eng' else row['text'], axis=1)

CPU times: total: 1h 27min 27s
Wall time: 6h 36min 1s


In [ ]:
# %%time

# data_all['text_with_tags_ru'] = data_all.apply(lambda row: en_ru_ext(row['text_with_tags']) if row['language']=='eng' else row['text'], axis=1)

In [ ]:
data_all.to_excel('data_all_with_tags_extention.xlsx')

In [147]:
%%time

text_with_tags_ru_list = []

for text_with_tags in tqdm(data_all[(~data_all.text_with_tags.isna())&(data_all.language=='eng')].text_with_tags.values.tolist()):
    
    if len(text_with_tags)>=5000:
        text_with_tags_ru_list.append(en_ru_ext(text_with_tags))
    else:
        text_with_tags_ru_list.append(en_ru(text_with_tags))

100%|██████████| 3029/3029 [3:40:41<00:00,  4.37s/it]  

CPU times: total: 1h 23min 23s
Wall time: 3h 40min 41s


In [150]:
len(text_with_tags_ru_list)

3029

In [149]:
pickle.dump(text_with_tags_ru_list, open("text_with_tags_ru_list.p", "wb"))

### Очистка от мусора

In [357]:
data = pd.read_excel('data_all_with_tags_extention.xlsx', index_col=0)

In [358]:
print(data.shape)

data = data.drop_duplicates()

print(data.shape)

(5242, 13)
(5242, 13)


In [359]:
#Неправильные ссылки. В коде исправлено.

data.href = np.where(data.source=='prc_roday', np.nan, data.href)

In [360]:
text_with_tags_ru_list = pickle.load(open("text_with_tags_ru_list.p", 'rb'))

In [361]:
len(text_with_tags_ru_list)

3029

In [362]:
data.isna().sum()

source               0
country              0
language             0
title                0
date                 0
href              1528
text                 0
text_with_tags    1555
year                 0
type                 0
category             0
title_ru             0
text_ru              0
dtype: int64

In [363]:
#Добавялем перевод тэгов

In [364]:
data.shape

(5242, 13)

In [365]:
data[(~data.text_with_tags.isna())&(data.language=='eng')].shape

(3029, 13)

In [366]:
data[(data.text_with_tags.isna())|(data.language!='eng')].shape

(2213, 13)

In [367]:
2213 + 3029

5242

In [368]:
data_1 = data[(~data.text_with_tags.isna())&(data.language=='eng')]

data_1['text_with_tags_ru'] = text_with_tags_ru_list

C:\Users\User\AppData\Local\Temp\ipykernel_15224\333435675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1['text_with_tags_ru'] = text_with_tags_ru_list


In [369]:
data_2 = data[(data.text_with_tags.isna())|(data.language!='eng')]

data_2['text_with_tags_ru'] = data_2['text_with_tags']

C:\Users\User\AppData\Local\Temp\ipykernel_15224\3031623675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2['text_with_tags_ru'] = data_2['text_with_tags']


In [370]:
data = pd.concat([data_1, data_2], axis=0)

In [371]:
#Для текстовой обработки
data.text_with_tags_ru.fillna('', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_15224\1255075864.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.text_with_tags_ru.fillna('', inplace=True)


In [372]:
data.shape

(5242, 14)

In [373]:
def replace_text(text, pattern):
    """Убираем предложения в искомой фразой и после"""
    clear_text = []
    
    for i in text.split('.'):
        if pattern not in i:
            clear_text.append(i)
        else:
            break
    return ''.join(clear_text)

pattern = 'Если вам понравились новости Китая'

data.text_ru = data.text_ru.apply(lambda value: replace_text(value, pattern))

In [374]:
ban_words = [
    'Сайти',
    'и мы свяжемся с вами в течение 15 минут',
    'коронавирус',
    'ковид',
    'порно',
    'covid',
    'тюрьма',
    'насилие',
    'секс',
    'Добрый день, уважаемые клиенты',
    'Откройте для себя улыбку своей мечты в самом сердце Шанхая!'
]

print(data.shape)
print()

for word in ban_words:
    data = data[~data.text_ru.str.contains(word)]
    data = data[~data.title_ru.str.contains(word)]
    print(word, data.shape)
    print()

(5242, 14)

Сайти (5242, 14)

и мы свяжемся с вами в течение 15 минут (5240, 14)

коронавирус (5212, 14)

ковид (5207, 14)

порно (5183, 14)

covid (5183, 14)

тюрьма (5181, 14)

насилие (5176, 14)

секс (5171, 14)

Добрый день, уважаемые клиенты (5170, 14)

Откройте для себя улыбку своей мечты в самом сердце Шанхая! (5169, 14)



In [375]:
replace_words = [
    'Всем привет! ',
    'Содержание:',
    'Портал PRC.TODAY – ',
    'Портал PRC.TODAY (Китай сегодня) – ',
    'Портал PRC.TODAY (Китай сегодня) – ',
    'Портал PRC.TODAY (Китай сегодня)  – ',
    'ОБНОВЛЕНИЕ: ',
    'Vietnam Briefing отслеживает все, что происходит в деловых и экономических новостях Вьетнама, так что вам не придется это делать. Вот что произошло на этой неделе.',
    'Эмиль Авдалиани',
    'Автор: Эмиль Авдалиани',
    'Автор: Shrakat ',
    '💫 ',
    '🌟 '
]

for word in replace_words:
    data.text_ru = data.text_ru.apply(lambda value: value.replace(word, ""))
    data.text_with_tags_ru = data.text_with_tags_ru.apply(lambda value: value.replace(word, ""))

In [376]:
data.title_ru = data.title_ru.apply(lambda value: value.replace("💫 ", ""))
data.title_ru = data.title_ru.apply(lambda value: value.replace("🌟 ", ""))

In [377]:
data.text_ru = data.text_ru.apply(lambda value: value.strip())
data.title_ru = data.title_ru.apply(lambda value: value.strip())

In [378]:
#Убираем лишние пробелы
data.text = data.text.apply(lambda value: re.sub(" +", " ", value))
data.title = data.title.apply(lambda value: re.sub(" +", " ", value))
data.text_ru = data.text_ru.apply(lambda value: re.sub(" +", " ", value))
data.title_ru = data.title_ru.apply(lambda value: re.sub(" +", " ", value))

In [379]:
data[data.text==''].shape

(0, 14)

In [380]:
data[data.title==''].shape

(0, 14)

In [381]:
# Джойним ключевые слова. В этой итерации они будут не везде, так как данные были обновлены. 
# В прошлой версии отдавал Виталию, чтобы он запустил у себя эту частью.
# Поэтому в следующей итерации эту часть необходимо заменить на подбор ключевых слов.

In [382]:
keywords = pd.read_excel('old_data_with_keywords.xlsx', index_col=0)[['title','date','keywords']]

In [383]:
print(keywords.shape)

keywords = keywords.drop_duplicates()

print(keywords.shape)

(3867, 3)
(3863, 3)


In [384]:
keywords.head(3)

,title,date,keywords
0,Торговля на OZON и Wildberries: Как начать про...,2023-12-27,"покупатели, полезно, помочь, найти, внимательн..."
1,Оптимизация процесса доставки товаров с 1688 д...,2023-12-20,"увеличение, компании, мы, бизнес, рынок, стран..."
2,Китайский Новый год: подготовьтесь к праздничн...,2023-12-19,"культура, закрытие, год, последний, месяц, нов..."


In [385]:
keywords = keywords.rename(columns={'title': 'title_ru'})

In [386]:
data.date = data.date.astype('str')
keywords.date = keywords.date.astype('str')

In [387]:
data = data.merge(keywords, how='left', on=['title_ru','date'])

data.keywords.fillna(np.nan, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_15224\371841712.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.keywords.fillna(np.nan, inplace=True)


In [389]:
data.columns

Index(['source', 'country', 'language', 'title', 'date', 'href', 'text',
       'text_with_tags', 'year', 'type', 'category', 'title_ru', 'text_ru',
       'text_with_tags_ru', 'keywords'],
      dtype='object')

In [390]:
right_order = [
    'source', 
    'country', 
    'language', 
    'date', 
    'category', 
    'type', 
    'href',
    'title', 
    'text',
    'text_with_tags', 
    'title_ru', 
    'text_ru',
    'text_with_tags_ru', 
    'keywords'
]

In [391]:
data = data[right_order]

In [392]:
data.head()

,source,country,language,date,category,type,href,title,text,text_with_tags,title_ru,text_ru,text_with_tags_ru,keywords
0,india-briefing,India,eng,2024-06-27 14:43:05,economic,news,https://www.india-briefing.com/news/india-stan...,India’s Standing in the Emerging Asia Manufact...,According to the Emerging Asia Manufacturing I...,<p><strong><em>According to the Emerging Asia ...,Положение Индии в Индексе производства развива...,Согласно Индексу производства развивающихся ст...,<p><strong><em>Согласно Индексу производственн...,NaN
1,india-briefing,India,eng,2024-06-26 15:37:11,economic,news,https://www.india-briefing.com/news/fdi-landsc...,FDI in India FY 2023-24: Top State and Sector ...,We discuss which Indian states and sectors rec...,<p><strong><em>We discuss which Indian states ...,ПИИ в Индию в 2023-2024 финансовом году: основ...,"Мы обсуждаем, какие индийские штаты и отрасли ...","<p><strong><em>Мы обсуждаем, какие индийские ш...",NaN
2,india-briefing,India,eng,2024-06-13 20:17:36,economic,news,https://www.india-briefing.com/news/india-to-p...,India to Propel South Asia as Fastest Growing ...,The World Bank projects that the Indian econom...,<p><em><strong>The World Bank projects that th...,Индия будет продвигать Южную Азию как самый бы...,"По прогнозам Всемирного банка, индийская эконо...","<p><em><strong>По прогнозам Всемирного банка, ...",NaN
3,india-briefing,India,eng,2024-06-13 19:30:33,economic,news,https://www.india-briefing.com/news/india-qata...,India-Qatar Investment and Trade Relations,India and Qatar have built a robust investment...,<p>India and Qatar have built a robust investm...,Индийско-Катарские инвестиционные и торговые о...,Индия и Катар установили прочные инвестиционны...,<p>Индия и Катар установили прочные инвестицио...,NaN
4,india-briefing,India,eng,2024-06-11 10:30:14,economic,news,https://www.india-briefing.com/news/modi-2024-...,Ministerial Roles in Modi’s New Cabinet: Full ...,Bharatiya Janata Party (BJP) leader Narendra M...,<p><em><strong>Bharatiya Janata Party (BJP) le...,Министерские роли в новом кабинете Моди: полны...,Лидер партии «Бхаратия Джаната» (БДП) Нарендра...,<p><em><strong>Лидер партии «Бхаратия Джаната»...,NaN


In [393]:
data.shape

(5171, 14)

In [394]:
data.isna().sum()/data.shape[0]*100

source                0.000000
country               0.000000
language              0.000000
date                  0.000000
category              0.000000
type                  0.000000
href                 28.988590
title                 0.000000
text                  0.000000
text_with_tags       29.510733
title_ru              0.000000
text_ru               0.000000
text_with_tags_ru     0.000000
keywords             46.219300
dtype: float64

In [395]:
data.to_excel('Межд источники Хозин v4.xlsx')